In [1]:
import tensorflow as tf
import mnist_reader
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.datasets import fetch_20newsgroups
from sklearn.datasets import fetch_mldata
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
import math
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from scipy import sparse,spatial
import kmeans
from sklearn.neighbors import KNeighborsClassifier
import csv
%matplotlib inline

/Users/sasankauppu/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
def show_images(images, cols = 1, titles = None):
    """Display a list of images in a single figure with matplotlib.
    
    Parameters
    ---------
    images: List of np.arrays compatible with plt.imshow.
    
    cols (Default = 1): Number of columns in figure (number of rows is 
                        set to np.ceil(n_images/float(cols))).
    
    titles: List of titles corresponding to each image. Must have
            the same length as titles.
    """
    assert((titles is None)or (len(images) == len(titles)))
    n_images = len(images)
    if titles is None: titles = ['Image (%d)' % i for i in range(1,n_images + 1)]
    fig = plt.figure()
    for n, (image, title) in enumerate(zip(images, titles)):
        a = fig.add_subplot(cols, np.ceil(n_images/float(cols)), n + 1)
        if image.ndim == 2:
            plt.gray()
        plt.imshow(image.reshape(28,28))
        a.set_title(title)
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_images)
    plt.show()

In [6]:
#@citation:https://stackoverflow.com/a/43538941
class Dataset:
    def __init__(self,data):
        self._index_in_epoch = 0
        self._epochs_completed = 0
        self._data = data
        self._num_examples = data.shape[0]
        pass

    @property
    def data(self):
        return self._data

    def next_batch(self,batch_size,shuffle = True):
        start = self._index_in_epoch
        if start == 0 and self._epochs_completed == 0:
            idx = np.arange(0, self._num_examples)  # get all possible indexes
            np.random.shuffle(idx)  # shuffle indexe
            self._data = self.data[idx]  # get list of `num` random samples

        # go to the next batch
        if start + batch_size > self._num_examples:
            self._epochs_completed += 1
            rest_num_examples = self._num_examples - start
            data_rest_part = self.data[start:self._num_examples]
            idx0 = np.arange(0, self._num_examples)  # get all possible indexes
            np.random.shuffle(idx0)  # shuffle indexes
            self._data = self.data[idx0]  # get list of `num` random samples

            start = 0
            self._index_in_epoch = batch_size - rest_num_examples #avoid the case where the #sample != integar times of batch_size
            end =  self._index_in_epoch  
            data_new_part =  self._data[start:end]  
            return np.concatenate((data_rest_part, data_new_part), axis=0)
        else:
            self._index_in_epoch += batch_size
            end = self._index_in_epoch
            return self._data[start:end]

In [7]:
def encoder(x,weights,biases,layers=2):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),biases['encoder_b1']))
    if layers==1:
        return layer_1
    
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']),biases['encoder_b2']))
    return layer_2


def decoder(x,weights,biases,layers=2):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),biases['decoder_b1']))
    if layers==1:
        return layer_1
    
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']),biases['decoder_b2']))
    return layer_2

In [8]:
def AutoEncoder(dataset,img_data=False,layers=2,num_hidden_1=None,num_hidden_2=None):
    learning_rate = 0.01
    batch_size = 256
    num_steps = 30000
    
    num_input = dataset.shape[1]
    dsobj=Dataset(dataset)

    # tf Graph input (only pictures)
    X = tf.placeholder("float", [None, num_input])
    
    if num_hidden_2 == None:
        weights = {
        'encoder_h1': tf.Variable(tf.random_normal([num_input, num_hidden_1])),
        'decoder_h1': tf.Variable(tf.random_normal([num_hidden_1, num_input])),
        }
        biases = {
        'encoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
        'decoder_b1': tf.Variable(tf.random_normal([num_input])),
        }
    else:
        weights = {
        'encoder_h1': tf.Variable(tf.random_normal([num_input, num_hidden_1])),
        'encoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_hidden_2])),
        'decoder_h1': tf.Variable(tf.random_normal([num_hidden_2, num_hidden_1])),
        'decoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_input])),
        }
        biases = {
        'encoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
        'encoder_b2': tf.Variable(tf.random_normal([num_hidden_2])),
        'decoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
        'decoder_b2': tf.Variable(tf.random_normal([num_input])),
        }
    
    # Construct model
    encoder_op = encoder(X,weights,biases,layers)
    decoder_op = decoder(encoder_op,weights,biases,layers)

    # Prediction
    y_pred = decoder_op
    # Targets (Labels) are the input data.
    y_true = X

    # Define loss and optimizer, minimize the squared error
    loss_op = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
    optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss_op)

    # Initialize the variables (i.e. assign their default value)
    init = tf.global_variables_initializer()

    sess = tf.Session()
    sess.run(init)

    # Training
    prevl=0
    if img_data:
        oneimg=[]
        oneimg.append(dataset[1450])
    for i in range(num_steps):
        # Get the next batch of MNIST data (only images are needed, not labels)
        batch_x = dsobj.next_batch(batch_size)

        # Run optimization op (backprop) and cost op (to get loss value)
        _, loss = sess.run([optimizer, loss_op], feed_dict={X: batch_x})
        
        # Display progress
        if i % 1000 == 0:
            print('Step %i: Minibatch Loss: %f' % (i, loss))
            features,y_pred,loss = sess.run((encoder_op,decoder_op,loss_op), feed_dict={X: dataset})
            if img_data:
                oneimg.append(y_pred[1450])
        
        
        prevl=loss
            
    features,y_pred,loss = sess.run((encoder_op,decoder_op,loss_op), feed_dict={X: dataset})
    if img_data:
        oneimg.append(y_pred[1450])
        show_images(oneimg,3)
    
    return y_pred

In [18]:
mnist = fetch_mldata('MNIST original', data_home="/Users/sasankauppu/Desktop/Data Mining CS6220/")
mnistdata = mnist.data/255.0

In [19]:
y_pred = AutoEncoder(mnistdata,img_data=True,layers=2,num_hidden_1=392,num_hidden_2=256)
acc,gini=kmeans.runKmeans(k=10,X=y_pred,y=mnist.target,nl=10,cosine=True)

Step 0: Minibatch Loss: 0.458211
Step 1000: Minibatch Loss: 0.146040
Step 2000: Minibatch Loss: 0.137453
Step 3000: Minibatch Loss: 0.125513
Step 4000: Minibatch Loss: 0.119121
Step 5000: Minibatch Loss: 0.119424
Step 6000: Minibatch Loss: 0.113651
Step 7000: Minibatch Loss: 0.112512
Step 8000: Minibatch Loss: 0.111941
Step 9000: Minibatch Loss: 0.110952
Step 10000: Minibatch Loss: 0.106194
Step 11000: Minibatch Loss: 0.103793
Step 12000: Minibatch Loss: 0.102372
Step 13000: Minibatch Loss: 0.101006
Step 14000: Minibatch Loss: 0.097676
Step 15000: Minibatch Loss: 0.098996
Step 16000: Minibatch Loss: 0.097703
Step 17000: Minibatch Loss: 0.093787
Step 18000: Minibatch Loss: 0.093939
Step 19000: Minibatch Loss: 0.091941
Step 20000: Minibatch Loss: 0.091331
Step 21000: Minibatch Loss: 0.089706
Step 22000: Minibatch Loss: 0.089069
Step 23000: Minibatch Loss: 0.087355
Step 24000: Minibatch Loss: 0.089203
Step 25000: Minibatch Loss: 0.087961
Step 26000: Minibatch Loss: 0.088422
Step 27000: Mi

round:  0  number of shuffles:  70000
round:  10  number of shuffles:  740
round:  20  number of shuffles:  314
round:  30  number of shuffles:  70
round:  40  number of shuffles:  2
round:  41  number of shuffles:  0


In [31]:
ind = np.arange( y_pred.shape[ 0 ] )
np.random.shuffle( ind )
y_pred_n = y_pred[ ind[ :10000 ], : ]
y_actual_n = mnistdata[ ind[ :10000 ], : ]

lab_n = mnist.target[ ind[ :10000 ]]

In [32]:
neigh = KNeighborsClassifier(n_neighbors=20,metric="cosine")
neigh.fit(y_actual_n, lab_n) 

neigh1 = KNeighborsClassifier(n_neighbors=20,metric="cosine")
neigh1.fit(y_pred_n, lab_n) 

nbors = neigh.kneighbors(n_neighbors=20)[1]
nbors1 = neigh1.kneighbors(n_neighbors=20)[1]
f=0.0
for i in range(10000):
    f+= (len(set(nbors[i]).intersection(set(nbors1[i]))))/20.0
    
print "% intersections in a random sample ",f/10000

% intersections in a random sample  0.523985


In [2]:
fashion_X, fashion_y = mnist_reader.load_mnist('data/', kind='train')
fashion_X1, fashion_y1 = mnist_reader.load_mnist('data/', kind='t10k')

fashion_X = np.concatenate((fashion_X,fashion_X1))
fashion_y = np.concatenate((fashion_y,fashion_y1))

del (fashion_X1)
del (fashion_y1)

fashion_X=fashion_X/255.0

In [9]:
y_pred = AutoEncoder(fashion_X,img_data=True,layers=2,num_hidden_1=392,num_hidden_2=256)

Step 0: Minibatch Loss: 0.399276
Step 1000: Minibatch Loss: 0.154146
Step 2000: Minibatch Loss: 0.135699
Step 3000: Minibatch Loss: 0.138899
Step 4000: Minibatch Loss: 0.123851
Step 5000: Minibatch Loss: 0.130656
Step 6000: Minibatch Loss: 0.123569
Step 7000: Minibatch Loss: 0.123618
Step 8000: Minibatch Loss: 0.127791
Step 9000: Minibatch Loss: 0.118792
Step 10000: Minibatch Loss: 0.122539
Step 11000: Minibatch Loss: 0.115564
Step 12000: Minibatch Loss: 0.122926
Step 13000: Minibatch Loss: 0.120640
Step 14000: Minibatch Loss: 0.113497
Step 15000: Minibatch Loss: 0.112509
Step 16000: Minibatch Loss: 0.108649
Step 17000: Minibatch Loss: 0.114052
Step 18000: Minibatch Loss: 0.108014
Step 19000: Minibatch Loss: 0.104601
Step 20000: Minibatch Loss: 0.109074
Step 21000: Minibatch Loss: 0.103903
Step 22000: Minibatch Loss: 0.108101
Step 23000: Minibatch Loss: 0.110171
Step 24000: Minibatch Loss: 0.115304
Step 25000: Minibatch Loss: 0.110545
Step 26000: Minibatch Loss: 0.109926
Step 27000: Mi

In [10]:
ind = np.arange( y_pred.shape[ 0 ] )
np.random.shuffle( ind )
y_pred_n = y_pred[ ind[ :20000 ], : ]
y_actual_n = fashion_X[ ind[ :20000 ], : ]

lab_n = fashion_y[ ind[ :20000 ]]

In [11]:
neigh = KNeighborsClassifier(n_neighbors=20,metric="cosine")
neigh.fit(y_actual_n, lab_n) 

neigh1 = KNeighborsClassifier(n_neighbors=20,metric="cosine")
neigh1.fit(y_pred_n, lab_n) 

nbors = neigh.kneighbors(n_neighbors=20)[1]
nbors1 = neigh1.kneighbors(n_neighbors=20)[1]
f=0.0
for i in range(20000):
    f+= (len(set(nbors[i]).intersection(set(nbors1[i]))))/20.0
    
print "% intersections in a random sample ",f/10000

% intersections in a random sample  0.4692
